# Effect of Sparsification on Graph Topology

This notebook analyzes how different sparsification methods (Random, Jaccard, Adamic-Adar, Effective Resistance, Metric Backbone) affect the topological properties of the graph, such as degree distribution, density, and connectivity.

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent.parent))

from src import (
    DatasetLoader,
    GraphSparsifier,
    set_global_seed,
    compute_graph_stats,
    print_text_table,
    calculate_jaccard_scores,
    calculate_adamic_adar_scores
)
from src.sparsification.metrics import calculate_approx_effective_resistance_scores
from src.sparsification.metric_backbone import metric_backbone_sparsify
from torch_geometric.utils import to_scipy_sparse_matrix, sort_edge_index

set_global_seed(42)

if torch.cuda.is_available():
    DEVICE = "cuda"
elif torch.backends.mps.is_available():
    DEVICE = "mps"
else:
    DEVICE = "cpu"

print(f"Using device: {DEVICE}")

## Setup and Imports

We begin by importing the necessary libraries and setting up our computational environment. We configure the device (GPU if available, otherwise CPU) and set a random seed for reproducibility.

## Dataset Loading

We load three benchmark graph datasets (Cora, Flickr, PubMed) that are commonly used in graph neural network research. These datasets vary in size and structure, allowing us to evaluate how sparsification methods generalize across different graph topologies.

In [ ]:
loader = DatasetLoader(root="../data")

dataset_names = ["cora", "flickr", "pubmed"]
datasets = {}

for name in dataset_names:
    data, _, _ = loader.get_dataset(name, DEVICE)
    datasets[name] = data
    print(f"Loaded {name.capitalize()}: {data.num_nodes:,} nodes, {data.edge_index.shape[1]:,} edges")

print(f"\nTotal datasets loaded: {len(datasets)}")

### Dataset Characteristics

The datasets have been successfully loaded. Each dataset represents a different scale and domain:
- **Cora**: A citation network with moderate size
- **Flickr**: A social network with higher connectivity
- **PubMed**: A larger citation network with different structural properties

These variations allow us to assess how sparsification methods perform across different graph characteristics.

## Sparsification Analysis Function

We define a function to analyze how graph topology changes under different sparsification strategies. The function:
1. Computes baseline topological statistics (degree distribution, density, connectivity)
2. Calculates edge importance scores using the specified method:
   - **Random**: Uniformly random scores (baseline)
   - **Jaccard**: Similarity based on neighborhood overlap
   - **Adamic-Adar**: Weighted by inverse log-degree of common neighbors
   - **Effective Resistance**: Electrical distance measuring criticality of edges (bridges have high resistance)
   - **Metric Backbone**: Removes redundant edges using the Relaxed Triangle Inequality
3. Progressively removes edges based on these scores at various retention rates
4. Tracks how topological properties evolve as the graph becomes sparser

Note: Metric Backbone uses a different approach (alpha parameter) rather than retention rates, so it's handled separately.

In [ ]:
def analyze_sparsification_topology(data, method, retention_rates):
    """
    Sparsifies the graph at different rates and computes topological stats.
    """
    results = {}
    
    results["Original"] = compute_graph_stats(data)
    
    if method == 'random':
        scores = torch.rand(data.edge_index.shape[1], device=DEVICE)
        edge_index = data.edge_index
        
    elif method == 'jaccard':
        edge_index_cpu = data.edge_index.cpu()
        adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
        
        scores_np = calculate_jaccard_scores(adj)
        
        adj.data = scores_np
        adj_coo = adj.tocoo()
        
        row = torch.from_numpy(adj_coo.row).to(torch.long).to(DEVICE)
        col = torch.from_numpy(adj_coo.col).to(torch.long).to(DEVICE)
        edge_index = torch.stack([row, col], dim=0)
        scores = torch.from_numpy(adj_coo.data).to(torch.float).to(DEVICE)
        
    elif method == 'adamic-adar':
        edge_index_cpu = data.edge_index.cpu()
        adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
        
        scores_np = calculate_adamic_adar_scores(adj)
        
        adj.data = scores_np
        adj_coo = adj.tocoo()
        
        row = torch.from_numpy(adj_coo.row).to(torch.long).to(DEVICE)
        col = torch.from_numpy(adj_coo.col).to(torch.long).to(DEVICE)
        edge_index = torch.stack([row, col], dim=0)
        scores = torch.from_numpy(adj_coo.data).to(torch.float).to(DEVICE)
        
    elif method == 'approx-effective-resistance':
        edge_index_cpu = data.edge_index.cpu()
        adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
        
        # Use approximate effective resistance (JL-projection based, O(m log n))
        scores_np = calculate_approx_effective_resistance_scores(adj)
        
        adj.data = scores_np
        adj_coo = adj.tocoo()
        
        row = torch.from_numpy(adj_coo.row).to(torch.long).to(DEVICE)
        col = torch.from_numpy(adj_coo.col).to(torch.long).to(DEVICE)
        edge_index = torch.stack([row, col], dim=0)
        scores = torch.from_numpy(adj_coo.data).to(torch.float).to(DEVICE)
        
    else:
        raise ValueError(f"Unknown method: {method}")
        
    for rate in retention_rates:
        k = int(rate * edge_index.shape[1])
        
        if k > 0:
            _, indices = torch.topk(scores, k)
            subset_edge_index = edge_index[:, indices]
        else:
            subset_edge_index = torch.empty((2, 0), dtype=torch.long, device=DEVICE)
            
        class SparseGraph:
            def __init__(self, edge_index, num_nodes):
                self.edge_index = edge_index
                self.num_nodes = num_nodes
        
        sparse_data = SparseGraph(subset_edge_index, data.num_nodes)
        results[f"Ret {rate:.1f}"] = compute_graph_stats(sparse_data)
    return results

def analyze_metric_backbone_topology(data, alpha_values):
    """
    Sparsifies the graph using metric backbone with different alpha values.
    Uses Jaccard distance (1 - Jaccard similarity) as the edge metric.
    """
    results = {}
    
    results["Original"] = compute_graph_stats(data)
    
    edge_index_cpu = data.edge_index.cpu()
    adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
    
    jaccard_sim = calculate_jaccard_scores(adj)
    jaccard_dist = 1.0 - jaccard_sim
    
    for alpha in alpha_values:
        sparse_data, stats = metric_backbone_sparsify(data, jaccard_dist, alpha=alpha, verbose=False)
        retention_rate = stats['retention_ratio']
        results[f"α={alpha:.1f} ({retention_rate:.1%})"] = compute_graph_stats(sparse_data)
    
    return results

## Running Sparsification Experiments

We now systematically apply five sparsification methods across all datasets:
- **Random**: Removes edges uniformly at random (baseline)
- **Jaccard**: Preserves edges between nodes with high neighborhood overlap
- **Adamic-Adar**: Prioritizes edges connecting nodes through low-degree intermediaries
- **Effective Resistance**: Retains edges with high electrical resistance (bridges, bottlenecks)
- **Metric Backbone**: Uses the Relaxed Triangle Inequality to remove redundant edges where efficient detours exist

For the first four methods, we test retention rates from 90% down to 10% in 10% increments. For Metric Backbone, we vary the alpha parameter (stretch factor) which controls the redundancy threshold.

In [ ]:
methods = ["random", "jaccard", "adamic-adar", "approx-effective-resistance"] 
retention_rates = [1-0.1*i for i in range(1,10)]
alpha_values = [1.0, 1.5, 2.0, 3.0, 5.0, 10.0]

all_results = {} 

for name, data in datasets.items():
    all_results[name] = {}
    print(f"\n{'='*30}\n DATASET: {name.capitalize()}\n{'='*30}")
    
    for method in methods:
        print(f"\nRunning {method.capitalize()} sparsification...")
        try:
            stats = analyze_sparsification_topology(data, method, retention_rates)
            all_results[name][method] = stats
            
            print_text_table(
                stats, 
                title=f"{name.capitalize()} - {method.capitalize()} Topology", 
                col_width=12
            )
        except Exception as e:
            print(f"Failed to run {method} on {name}: {e}")
    
    print(f"\nRunning Metric Backbone sparsification...")
    try:
        stats = analyze_metric_backbone_topology(data, alpha_values)
        all_results[name]['metric-backbone'] = stats
        
        print_text_table(
            stats,
            title=f"{name.capitalize()} - Metric Backbone Topology",
            col_width=15
        )
    except Exception as e:
        print(f"Failed to run metric-backbone on {name}: {e}")

### Topological Statistics Summary

The tables above show how key graph properties evolve under sparsification:

**Key Observations:**
- **Average Degree**: Decreases approximately linearly with retention rate for threshold-based methods (Random, Jaccard, Adamic-Adar, Effective Resistance)
- **Density**: Becomes sparser as edges are removed, particularly noticeable in denser graphs
- **Connected Components**: May fragment as critical bridging edges are removed
- **Method Differences**: Structural methods (Jaccard, Adamic-Adar, Effective Resistance) tend to preserve connectivity better than random removal at equivalent retention rates
- **Effective Resistance**: Identifies critical edges (bridges) with high resistance; preserving these maintains graph connectivity
- **Metric Backbone**: Operates differently by identifying redundant edges based on the triangle inequality. Lower alpha values are more aggressive, removing edges where efficient detours exist. The retention ratio varies based on graph structure.

These results demonstrate that not all edges are equal for maintaining graph structure—intelligently selecting which edges to retain can preserve topological integrity better than random pruning. Metric Backbone provides a principled geometric approach by identifying edges that are "shortcuts" in the graph's metric space.

## Visualizing Degree Evolution

We visualize how the average node degree changes as we progressively remove edges. This plot helps us understand:
- The rate of structural degradation under each method
- Whether different methods preserve degree distribution differently
- How dataset characteristics influence sparsification behavior

Note: Metric Backbone is not included in this plot as it uses a different parameterization (alpha) rather than explicit retention rates.

In [ ]:
plt.figure(figsize=(18, 5))

for i, name in enumerate(dataset_names):
    plt.subplot(1, 3, i+1)
    plt.title(f"{name.capitalize()} - Avg Degree Decay", fontsize=14)
    
    for method in methods:
        if method not in all_results[name]:
            continue
            
        stats_dict = all_results[name][method]
        
        rates = []
        values = []
        
        for key, metrics in stats_dict.items():
            if key == "Original":
                r = 1.0
            else:
                r = float(key.split(" ")[1])
            
            rates.append(r)
            values.append(metrics["avg_degree"])
            
        sorted_pairs = sorted(zip(rates, values), reverse=True)
        xs, ys = zip(*sorted_pairs)
        
        plt.plot(xs, ys, marker='o', linewidth=2, label=method.capitalize())
        
    plt.xlabel("Retention Rate")
    plt.ylabel("Average Degree")
    plt.gca().invert_xaxis()
    plt.grid(True, alpha=0.3)
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.show()

### Degree Decay Analysis

The plots reveal that average degree decreases approximately linearly with retention rate across all methods and datasets. This is expected since we're removing a fixed proportion of edges. The similarity of curves across methods suggests that at the aggregate level, all three approaches remove edges at comparable rates—the key differences lie in *which* edges are removed, not *how many*.

## Edge Overlap Analysis

To understand whether different sparsification methods select similar or different edges, we compute the Jaccard similarity between the edge sets retained by each method pair. High overlap indicates the methods agree on which edges are important, while low overlap suggests they capture different aspects of graph structure.

In [ ]:
def get_edge_set(edge_index):
    """Converts edge_index to a set of undirected edges (u, v) where u < v."""
    src, dst = edge_index
    mask = src < dst
    edges = torch.stack([src[mask], dst[mask]], dim=1)
    return set(map(tuple, edges.cpu().numpy().tolist()))

def compute_method_scores(data, method):
    """Compute scores ONCE for a method - this is the expensive operation."""
    if method == 'random':
        return torch.rand(data.edge_index.shape[1], device=DEVICE), data.edge_index
    
    edge_index_cpu = data.edge_index.cpu()
    adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
    
    if method == 'jaccard':
        scores_np = calculate_jaccard_scores(adj)
    elif method == 'adamic-adar':
        scores_np = calculate_adamic_adar_scores(adj)
    elif method in ('effective-resistance', 'approx-effective-resistance'):
        scores_np = calculate_approx_effective_resistance_scores(adj)
    else:
        raise ValueError(f"Unknown method: {method}")
    
    adj.data = scores_np
    adj_coo = adj.tocoo()
    row = torch.from_numpy(adj_coo.row).to(torch.long).to(DEVICE)
    col = torch.from_numpy(adj_coo.col).to(torch.long).to(DEVICE)
    edge_index = torch.stack([row, col], dim=0)
    scores = torch.from_numpy(adj_coo.data).to(torch.float).to(DEVICE)
    return scores, edge_index

def get_sparsified_edges_from_scores(scores, edge_index, rate):
    """Get sparsified edges using precomputed scores (fast)."""
    k = int(rate * edge_index.shape[1])
    if k > 0:
        _, indices = torch.topk(scores, k)
        subset_edge_index = edge_index[:, indices]
    else:
        subset_edge_index = torch.empty((2, 0), dtype=torch.long, device=DEVICE)
    return get_edge_set(subset_edge_index)

# Generate pairs dynamically from methods list
pairs = [(methods[i], methods[j]) for i in range(len(methods)) for j in range(i+1, len(methods))]

plot_data = {}

for name, data in datasets.items():
    print(f"\n{'='*30}\n OVERLAP ANALYSIS: {name.capitalize()}\n{'='*30}")
    
    # Precompute scores ONCE per method (expensive part)
    print("Precomputing scores...")
    method_cache = {}
    for method in methods:
        print(f"  {method}...", end=" ", flush=True)
        method_cache[method] = compute_method_scores(data, method)
        print("done")
    
    dataset_overlaps = {}
    plot_data[name] = {f"{p[0]} vs {p[1]}": {'rates': [], 'overlaps': []} for p in pairs}
    
    # Now iterate over retention rates using cached scores (fast)
    for rate in retention_rates:
        edge_sets = {}
        for method in methods:
            scores, edge_index = method_cache[method]
            edge_sets[method] = get_sparsified_edges_from_scores(scores, edge_index, rate)
            
        rate_stats = {}
        for m1, m2 in pairs:
            set1 = edge_sets[m1]
            set2 = edge_sets[m2]
            
            if len(set1) == 0 and len(set2) == 0:
                jaccard_sim = 1.0
            else:
                intersection = len(set1.intersection(set2))
                union = len(set1.union(set2))
                jaccard_sim = intersection / union
            
            pair_key = f"{m1} vs {m2}"
            rate_stats[pair_key] = jaccard_sim
            plot_data[name][pair_key]['rates'].append(rate)
            plot_data[name][pair_key]['overlaps'].append(jaccard_sim)
            
        dataset_overlaps[f"Ret {rate:.1f}"] = rate_stats
        
    print_text_table(
        dataset_overlaps,
        title=f"{name.capitalize()} - Edge Overlap (Jaccard Similarity)",
        col_width=20,
        float_fmt="{:.2%}"
    )

plt.figure(figsize=(18, 5))

for i, name in enumerate(dataset_names):
    plt.subplot(1, 3, i+1)
    plt.title(f"{name.capitalize()} - Method Overlap", fontsize=14)
    
    for pair_key, data_points in plot_data[name].items():
        rates = data_points['rates']
        overlaps = data_points['overlaps']
        sorted_pairs = sorted(zip(rates, overlaps), reverse=True)
        xs, ys = zip(*sorted_pairs)
        
        plt.plot(xs, ys, marker='o', linewidth=2, label=pair_key)
        
    plt.xlabel("Retention Rate")
    plt.ylabel("Overlap")
    plt.gca().invert_xaxis()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.05)
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.show()

### Edge Selection Diversity

The overlap analysis reveals important insights about method similarity:

**Scientific Findings:**
- **Low overlap between Random and structural methods**: Random sparsification shows minimal agreement with Jaccard, Adamic-Adar, and Effective Resistance, confirming these structural methods capture non-random patterns
- **High overlap between similarity-based methods**: Jaccard and Adamic-Adar show high overlap, suggesting they both identify edges that preserve community structure
- **Effective Resistance distinctiveness**: Shows moderate overlap with similarity-based methods, as it prioritizes bridge edges and bottlenecks—a complementary criterion to neighborhood similarity
- **Retention rate dependency**: Overlap generally decreases at lower retention rates, indicating methods disagree more about which edges are "least important"

This demonstrates that structural sparsification methods capture meaningful topological patterns, making principled decisions about edge importance rather than arbitrary selection. Each method emphasizes different structural features: Jaccard/Adamic-Adar focus on community cohesion, while Effective Resistance identifies critical connectivity.

## Metric Backbone Comparison

We now analyze how Metric Backbone's edge selection compares to the other methods. Since Metric Backbone operates on a different parameter (alpha), we compare it at similar retention ratios achieved by different alpha values.

In [ ]:
mb_comparison_data = {}

# Methods to compare against (excluding random)
comparison_methods = ['jaccard', 'adamic-adar', 'approx-effective-resistance']

for name, data in datasets.items():
    print(f"\n{'='*30}\n METRIC BACKBONE COMPARISON: {name.capitalize()}\n{'='*30}")
    
    edge_index_cpu = data.edge_index.cpu()
    adj = to_scipy_sparse_matrix(edge_index_cpu, num_nodes=data.num_nodes).tocsr()
    jaccard_sim = calculate_jaccard_scores(adj)
    jaccard_dist = 1.0 - jaccard_sim
    
    # Precompute scores for each method once
    print("Precomputing method scores...")
    method_cache = {}
    for method in comparison_methods:
        print(f"  {method}...", end=" ", flush=True)
        method_cache[method] = compute_method_scores(data, method)
        print("done")
    
    mb_comparison_data[name] = {'alpha': [], 'retention': []}
    
    for method in comparison_methods:
        mb_comparison_data[name][f'overlap_with_{method}'] = []
    
    for alpha in alpha_values:
        sparse_data, stats = metric_backbone_sparsify(data, jaccard_dist, alpha=alpha, verbose=False)
        mb_edges = get_edge_set(sparse_data.edge_index)
        retention = stats['retention_ratio']
        
        mb_comparison_data[name]['alpha'].append(alpha)
        mb_comparison_data[name]['retention'].append(retention)
        
        target_rate = retention
        
        for method in comparison_methods:
            scores, edge_index = method_cache[method]
            method_edges = get_sparsified_edges_from_scores(scores, edge_index, target_rate)
            
            if len(mb_edges) == 0 and len(method_edges) == 0:
                overlap = 1.0
            else:
                intersection = len(mb_edges.intersection(method_edges))
                union = len(mb_edges.union(method_edges))
                overlap = intersection / union if union > 0 else 0.0
            
            mb_comparison_data[name][f'overlap_with_{method}'].append(overlap)
        
        print(f"α={alpha:.1f}: Retention={retention:.2%}, "
              f"Overlap with Jaccard={mb_comparison_data[name]['overlap_with_jaccard'][-1]:.2%}, "
              f"Overlap with Adamic-Adar={mb_comparison_data[name]['overlap_with_adamic-adar'][-1]:.2%}, "
              f"Overlap with Approx-ER={mb_comparison_data[name]['overlap_with_approx-effective-resistance'][-1]:.2%}")

plt.figure(figsize=(18, 5))

for i, name in enumerate(dataset_names):
    plt.subplot(1, 3, i+1)
    plt.title(f"{name.capitalize()} - Metric Backbone vs Others", fontsize=14)
    
    retentions = mb_comparison_data[name]['retention']
    
    plt.plot(retentions, mb_comparison_data[name]['overlap_with_jaccard'], 
             marker='o', linewidth=2, label='vs Jaccard')
    plt.plot(retentions, mb_comparison_data[name]['overlap_with_adamic-adar'], 
             marker='s', linewidth=2, label='vs Adamic-Adar')
    plt.plot(retentions, mb_comparison_data[name]['overlap_with_approx-effective-resistance'], 
             marker='^', linewidth=2, label='vs Approx-ER')
    
    plt.xlabel("Retention Rate")
    plt.ylabel("Edge Overlap (Jaccard)")
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.05)
    if i == 0:
        plt.legend()

plt.tight_layout()
plt.show()

### Metric Backbone Selection Analysis

The analysis reveals how Metric Backbone's geometric approach differs from other structural methods:

**Scientific Findings:**
- **Moderate overlap with similarity-based methods**: Metric Backbone shows partial agreement with Jaccard and Adamic-Adar, suggesting it captures some similar structural patterns while also identifying unique redundancies
- **Distinctive from Effective Resistance**: While Effective Resistance identifies critical bridge edges, Metric Backbone removes redundant edges where efficient detours exist—complementary but distinct geometric perspectives
- **Geometric vs. Similarity-based pruning**: While Jaccard/Adamic-Adar preserve edges with high neighborhood overlap, Metric Backbone removes edges where efficient detours exist through triangles—a fundamentally different criterion
- **Retention-dependent behavior**: The overlap varies with retention rate, indicating that at different sparsity levels, the methods prioritize different structural features
- **Complementary perspectives**: The distinct edge selections suggest these methods could be combined to capture multiple aspects of graph structure

This demonstrates that Metric Backbone provides a principled geometric perspective on edge importance, based on the Relaxed Triangle Inequality, which complements both the neighborhood-similarity perspective of Jaccard/Adamic-Adar and the connectivity-criticality perspective of Effective Resistance.

## Conclusion

This notebook systematically analyzed how different sparsification methods affect graph topology across multiple benchmark datasets. 

### Key Findings:

1. **Structural Preservation**: Structural methods (Jaccard, Adamic-Adar, Effective Resistance) preserve graph structure better than random removal at equivalent retention rates, maintaining connectivity and degree distribution more effectively.

2. **Similarity-based Convergence**: Jaccard and Adamic-Adar show high overlap in edge selection, suggesting they capture similar structural patterns—edges connecting nodes with many common neighbors.

3. **Effective Resistance Uniqueness**: Effective Resistance identifies critical bridge edges and bottlenecks, providing a complementary perspective focused on connectivity rather than community structure. It shows moderate overlap with similarity-based methods.

4. **Geometric Perspective**: Metric Backbone offers a fundamentally different approach based on the Relaxed Triangle Inequality. It identifies and removes redundant edges where efficient alternative paths exist, providing a complementary geometric view of edge importance.

5. **Dataset Generalization**: The relative behavior of methods is consistent across datasets of varying sizes and structures (citation networks, social networks), suggesting these findings generalize beyond specific graph types.

6. **Retention Rate Dependency**: As graphs become sparser (lower retention rates), methods diverge more in their edge selection, highlighting their different criteria for identifying "important" edges.

### Methodological Insights:

- **Community Cohesion** (Jaccard, Adamic-Adar): Preserve edges within densely connected clusters
- **Critical Connectivity** (Effective Resistance): Maintain bottleneck edges essential for graph connectivity
- **Geometric Efficiency** (Metric Backbone): Remove edges that violate the relaxed triangle inequality

### Implications for GNN Sparsification:

These topological analyses provide the foundation for understanding how sparsification will affect GNN performance. Methods that better preserve connectivity and community structure should maintain information flow in message-passing neural networks. The diversity of structural criteria (community cohesion, critical connectivity, geometric efficiency) suggests that different methods may be optimal for different graph types or downstream tasks. The next steps involve evaluating how these topological differences translate to downstream task performance.